# ReRank

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
# !pip install cohere 
# !pip install weaviate-client

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
import cohere
co = cohere.Client("c70AvJ6DUm1S6kwQJUyJsPEAI683RH2DjOr7QPiG")

In [4]:
import weaviate
auth_config = weaviate.auth.AuthApiKey(api_key = "76320a90-53d8-42bc-b41d-678647c6672e")

In [6]:
client = weaviate.Client(url = "https://cohere-demo.weaviate.network/",
                         auth_client_secret = auth_config,
                         additional_headers = {"X-Cohere-Api-Key":
                                               "c70AvJ6DUm1S6kwQJUyJsPEAI683RH2DjOr7QPiG",
                                              })
client.is_ready()

True

## Dense Retrieval

In [7]:
from utils import dense_retrieval

In [8]:
query = "How many World Cups have Brazil won?"

In [9]:
dense_retrieval_results = dense_retrieval(query, client)

In [10]:
from utils import print_result

In [11]:
print_result(dense_retrieval_results)

item 0
_additional:{'distance': -149.18587}

lang:en

text:Brazil is the most successful national team in the FIFA World Cup, being crowned winner five times: 1958, 1962, 1970, 1994 and 2002. The "Seleção" also has the best overall performance in the World Cup competition, both in proportional and absolute terms, with a record of 76 victories in 114 matches played, 129 goal difference, 247 points, and 19 losses. It is the only national team to have played in all World Cup editions without any absence nor need for playoffs, and the only team to have won the World Cup in four different continents: once in Europe (1958 Sweden), once in South America (1962 Chile), twice in North America (1970 Mexico and 1994 United States), and once in Asia (2002 South Korea/Japan). Brazil is also the most successful team in the now-defunct FIFA Confederations Cup, winning it four times, in 1997, 2005, 2009, and 2013.

title:Brazil national football team

url:https://en.wikipedia.org/wiki?curid=149286

vie

## Improving Keyword Search with ReRank

In [12]:
from utils import keyword_search

In [14]:
query_1 = "How many World Cups have Brazil won?"

In [15]:
query_1 = "How many World Cups have Brazil won?"
results = keyword_search(query_1, client,
                         properties = ["text", "title", "url", "views", "lang", "_additional {distance}"],
                         num_results = 3)

for i, result in enumerate(results):
    print(f"i: {i}")
    print(result.get("title"))
    print(result.get("text"))

i: 0
1950 FIFA World Cup
The 1950 FIFA World Cup was the fourth edition of the FIFA World Cup, the quadrennial international football championship for senior men's national teams and held in Brazil from 24 June to 16 July 1950. The planned 1942 and 1946 World Cups were cancelled due to World War II. This tournament ended the hiatus. Uruguay, who had won the inaugural competition in 1930, defeated the host nation, Brazil, in the deciding match of the four-team group of the final round. This was the only tournament not decided by a one-match final. It was also the inaugural tournament where the trophy was referred to as the Jules Rimet Cup, to mark the 25th anniversary of Jules Rimet's presidency of FIFA.
i: 1
1950 FIFA World Cup
The average attendance of nearly 61,000 per game, aided greatly by eight matches (including five featuring hosts Brazil) held in the newly built Maracanã, set a record that would not be broken until 1994. Not counting the Maracanã matches, the average attendance

In [16]:
query_1 = "How many World Cups have Brazil won?"
results = keyword_search(query_1, client,
                         properties = ["text", "title", "url", "views", "lang", "_additional {distance}"],
                         num_results = 500)

for i, result in enumerate(results):
    print(f"i: {i}")
    print(result.get("title"))
    #print(result.get('text'))

i: 0
1950 FIFA World Cup
i: 1
1950 FIFA World Cup
i: 2
Global North and Global South
i: 3
Brian Clough
i: 4
1950 FIFA World Cup
i: 5
1950 FIFA World Cup
i: 6
D.C. United
i: 7
1950 FIFA World Cup
i: 8
Marquinhos
i: 9
R.S.C. Anderlecht
i: 10
1950 FIFA World Cup
i: 11
Brazil national football team
i: 12
Brazil
i: 13
Brazil national football team
i: 14
The Tree of Life (film)
i: 15
Doughnut
i: 16
Marquinhos
i: 17
Brazil national football team
i: 18
D.C. United
i: 19
Booker Prize
i: 20
FIBA Basketball World Cup
i: 21
South Australia
i: 22
Genetic engineering
i: 23
R.S.C. Anderlecht
i: 24
University of Toronto
i: 25
1950 FIFA World Cup
i: 26
1974 FIFA World Cup
i: 27
Lacrosse
i: 28
FIFA U-20 World Cup
i: 29
Federalist Party
i: 30
New Age
i: 31
1994 FIFA World Cup
i: 32
FIFA World Cup hosts
i: 33
Jack Johnson (boxer)
i: 34
Split, Croatia
i: 35
Rugby World Cup
i: 36
Shia–Sunni relations
i: 37
FIFA World Cup
i: 38
Planet Nine
i: 39
Record producer
i: 40
1940 United States presidential election


In [17]:
def rerank_responses(query, responses, num_responses = 10):
    reranked_responses = co.rerank(model = "rerank-english-v2.0",
                                   query = query,
                                   documents = responses,
                                   top_n = num_responses,)
    return reranked_responses

In [18]:
texts = [result.get("text") for result in results]
reranked_text = rerank_responses(query_1, texts)

In [19]:
for i, rerank_result in enumerate(reranked_text):
    print(f"i: {i}")
    print(f"{rerank_result}")
    print()

i: 0
RerankResult<document['text']: Susana Polo of "Polygon" stated, "Fans have been asking Marvel to greenlight a movie with a female lead for years. But the film rights to most of Marvel Comics' most famous superheroines — Rogue, Storm, Jean Grey, Sue Storm and others — are held by 20th Century Fox. With a Black Widow movie backburnered until very recently, Carol Danvers' Captain Marvel is not just the most powerful superheroine Marvel Studios could have chosen, but the most notable one." Dorian Lynskey of "The Guardian" asserted, "The success of Ms Marvel and Captain Marvel has little to do with identity politics and everything to do with great storytelling. When Carol Danvers had her first solo book as Ms Marvel in 1977 (“This Female Fights Back!”), she was burdened with being Marvel’s token feminist role model: a superpowered Gloria Steinem." Mason Downey of "GameSpot" wrote, "You'd be hard pressed to find a more popular female superhero than Carol Danvers right now--and not just 

## Improving Dense Retrieval with ReRank

In [20]:
from utils import dense_retrieval

In [21]:
query_2 = "Who is the tallest person in history?"

In [22]:
results = dense_retrieval(query_2,client)

In [23]:
for i, result in enumerate(results):
    print(f"i: {i}")
    print(result.get("title"))
    print(result.get("text"))
    print()

i: 0
Robert Wadlow
Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri.

i: 1
Manute Bol
Bol came from a family of extraordinarily tall men and women. He said: "My mother was , my father , and my sister is . And my great-grandfather was even taller—." His ethnic group, the Dinka, and the Nilotic people of which they are a part, are among the tallest populations in the world. Bol's hometown, Turalei, is the origin of other exceptionally tall people, including basketball player Ring Ayuel. "I was born in a village, where you cannot measure yourself," Bol reflected. "I learned I was 7 foot 7 in 1979, when I was grown. I was about 18 or 19."

i: 2
Sultan Kösen
Sultan Kösen (born 10 December 1982) is a Turkish farmer who holds the Guinness World Record f

In [24]:
texts = [result.get("text") for result in results]
reranked_text = rerank_responses(query_2, texts)

In [25]:
for i, rerank_result in enumerate(reranked_text):
    print(f"i: {i}")
    print(f"{rerank_result}")
    print()

i: 0
RerankResult<document['text']: Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri., index: 0, relevance_score: 0.9734939>

i: 1
RerankResult<document['text']: Sultan Kösen (born 10 December 1982) is a Turkish farmer who holds the Guinness World Record for tallest living male at . Of Kurdish ethnicity, he is the seventh tallest man in history., index: 2, relevance_score: 0.8664718>

i: 2
RerankResult<document['text']: The Dutch are the tallest people in the world, by nationality, with an average height of for adult males and for adult females in 2009. The average height of young males in the Netherlands increased from 5 feet, 4 inches to approximately 6 feet between the 1850s until the early 2000s. People in the south are on average about short